In [ ]:
!pip install findspark
!pip install hdfs

In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/lib/spark'

In [9]:
!hdfs dfs -ls s3a://mlops-data/fraud-data

2023-08-23 12:38:58,519 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2023-08-23 12:38:58,600 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2023-08-23 12:38:58,600 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 40 items
-rw-rw-rw-   1 ubuntu ubuntu 2807409271 2022-12-05 09:23 s3a://mlops-data/fraud-data/2019-08-22.txt
-rw-rw-rw-   1 ubuntu ubuntu 2854479008 2022-12-05 09:24 s3a://mlops-data/fraud-data/2019-09-21.txt
-rw-rw-rw-   1 ubuntu ubuntu 2895460543 2022-12-05 09:27 s3a://mlops-data/fraud-data/2019-10-21.txt
-rw-rw-rw-   1 ubuntu ubuntu 2939120942 2022-12-05 09:30 s3a://mlops-data/fraud-data/2019-11-20.txt
-rw-rw-rw-   1 ubuntu ubuntu 2995462277 2022-12-05 09:32 s3a://mlops-data/fraud-data/2019-12-20.txt
-rw-rw-rw-   1 ubuntu ubuntu 2994906767 2022-12-05 09:35 s3a://mlops-data/fraud-data/2020-01-19.txt
-rw-rw-rw-   1 ubuntu ubuntu 2995431240 2022-12-05 09:38 s3a://mlops-data/fraud-data/2020-0

In [41]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import functions
from pyspark.sql.types import StringType, StructField, StructType

In [17]:
try:
    sc = SparkContext()
except ValueError:
    sc = SparkContext.getOrCreate()
    
if sc:
    sc.stop()
    
sc = SparkContext()

spark = SparkSession\
        .builder\
        .appName("mytestapp")\
        .getOrCreate()

In [8]:
import boto3
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import seaborn as sns
import warnings
import os
warnings.simplefilter('ignore')
s3 = boto3.resource('s3',
                endpoint_url='https://storage.yandexcloud.net',
                region_name = 'ru-central1',
                aws_access_key_id = 'YCAJEGyjDZ70buucdwAjU9B-c',
                aws_secret_access_key = 'YCPfQDBIwyDGS-U0AHlXFZcn59PTI30ZP1tCpGN4'
                )
data_bucket = s3.Bucket('mlops-data')

## Проверка подключения

In [9]:
count = sum(1 for _ in data_bucket.objects.all())
print(f'{count} файлов')

40 файлов


# Проверка корректности данных

## 1. Проверка полноты данных

In [46]:
def check_data(data, cols, n_bytes = 10000):
    txt = data.get().get('Body').read(n_bytes).decode()
    header = txt.split('\n')[0]
    body = '\n'.join(txt.split('\n')[1:])
    header = [i.strip() for i in header.replace('#', '').split('|')]
    df= pd.read_csv(io.StringIO(body),sep=',', engine = 'python', names=header)
    columns = df.columns.tolist()
    for new_c in columns:
        if new_c not in cols.keys():
            print(f'Обнаружен новый столбец {new_c}')
            cols[new_c] = dict()
            cols[new_c].update({'dtype': [df[new_c].dtype],
                                'NA': [df[new_c].isna().sum()],
                                'nrows': [df.shape[0]]
                                })
        else:
            cols[new_c]['dtype'].append(df[new_c].dtype)
            cols[new_c]['NA'].append(df[new_c].isna().sum())
            cols[new_c]['nrows'].append(df.shape[0])
    for old_c in cols.keys():
        if old_c not in columns:
            print(f'{old_c} нет в списке столбцов!')
    return cols # Dict[colname: [List of dtypes]]

def read_csv(s3obj):
    txt = s3obj.get().get('Body').read(1000).decode()
    header = txt.split('\n')[0]
    body = '\n'.join(txt.split('\n')[1:])
    header = [i.strip() for i in header.replace('#', '').split('|')]
    df = sql.read.text(os.path.join("s3a://" , i.bucket_name, i.key))
    header_ = df.limit(1)
    df = df.subtract(header_)
    df = df.toDF(header)
    return df

def read_csv(s3obj):
    #df = sql.read.text(os.path.join("s3a://" , i.bucket_name, i.key))
    df = sc.textFile(os.path.join("s3a://" , i.bucket_name, i.key))
    header =  df.first()
    df = df.filter(lambda line: line != header)
    header = [col.strip() for col in header.replace('#', '').split('|')]
    schema = StructType([StructField(str(i), StringType(), True) for i in header])
    return spark.createDataFrame(df, schema)

In [47]:
for i in data_bucket.objects.all():
    df = read_csv(i)
    break

In [48]:
df.show(5)

Py4JJavaError: An error occurred while calling o346.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 10.0 failed 4 times, most recent failure: Lost task 0.3 in stage 10.0 (TID 96, rc1b-dataproc-d-ponxn1lyzug6w6o0.mdb.yandexcloud.net, executor 5): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/hadoop/yarn/nm-local-dir/usercache/ubuntu/appcache/application_1692791613640_0005/container_1692791613640_0005_01_000008/pyspark.zip/pyspark/worker.py", line 605, in main
    process()
  File "/hadoop/yarn/nm-local-dir/usercache/ubuntu/appcache/application_1692791613640_0005/container_1692791613640_0005_01_000008/pyspark.zip/pyspark/worker.py", line 597, in process
    serializer.dump_stream(out_iter, outfile)
  File "/hadoop/yarn/nm-local-dir/usercache/ubuntu/appcache/application_1692791613640_0005/container_1692791613640_0005_01_000008/pyspark.zip/pyspark/serializers.py", line 271, in dump_stream
    vs = l

In [128]:
df = sql.read.text(os.path.join("s3a://" , i.bucket_name, i.key))
header =  df.limit(1)
df = df.subtract(header)

In [139]:
df.show(5)

+--------------------+
|               value|
+--------------------+
|13928259,2019-08-...|
|13928502,2019-08-...|
|13928629,2019-08-...|
|13928930,2019-08-...|
|13929200,2019-08-...|
+--------------------+
only showing top 5 rows



In [ ]:
df

In [132]:
df.limit(1).

DataFrame[value: string]

In [ ]:
[col.strip() for col in header.value.replace('#', '').split('|')]

In [66]:
df.printSchema()

root
 |-- # tranaction_id : string (nullable = true)
 |--  tx_datetime : string (nullable = true)
 |--  customer_id : string (nullable = true)
 |--  terminal_id : string (nullable = true)
 |--  tx_amount : string (nullable = true)
 |--  tx_time_seconds : string (nullable = true)
 |--  tx_time_days : string (nullable = true)
 |--  tx_fraud : string (nullable = true)
 |--  tx_fraud_scenario: string (nullable = true)



In [67]:
df.show(5)

+--------------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|    # tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+--------------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|0,2019-08-22 06:5...|         null|         null|         null|       null|             null|          null|      null|              null|
|1,2019-08-22 05:1...|         null|         null|         null|       null|             null|          null|      null|              null|
|2,2019-08-22 19:0...|         null|         null|         null|       null|             null|          null|      null|              null|
|3,2019-08-22 07:2...|         null|         null|         null|       null|             null|          null|      null|              null|
|4,2019-08-22 09:0..

In [ ]:
df = sql.read.parquet("s3a://yc-mdb-examples/dataproc/example01/set01")